In [ ]:
from funcs import funcs
from Crosswalk.Transformer import Transformer
from Crosswalk.DataCache import DataCache
from Crosswalk.NDAWriter import NDAWriter
from Crosswalk.Manager import Manager

from Crosswalk.Loader import Loader, BoxLoader, BoxHcdLoader, QintLoader, QintHcdLoader, RedcapLoader, KsadsLoader, ParentLoader
import pandas as pd
import numpy as np
from ccf.easy_yaml import EasyYaml
from ccf.redcap import RedcapTable



In [ ]:
#create folder for prepped structures, if it doesn't exist
!!mkdir prepped
!!mkdir prepped/hcd

In [ ]:
M = Manager(
        data =  DataCache(
            ParentLoader(),
            RedcapLoader('child'),
            RedcapLoader('teen'),
            BoxHcdLoader('PennCNP', 592325063896) ,
            QintHcdLoader() ,
            KsadsLoader()
        ),
        writer = NDAWriter(completed_dir="./prepped/hcd/", validator="vtcmd"),
        transformer = Transformer(funcs = funcs, map_dir='./maps/hcd/')
)


In [ ]:
# This step requires that you have a 'rosetta stone' file that has all the required NDA fields for 
# all subjects you intend to submit at this time.  This approach facilitates keeping track of subject counts
# across data types.  For example, if your required fields are already stored in XNAT because you had the CCF
# upload your imaging data for you, you can export this csv from XNAT and rename as appropriate.  
# Place this file at the main level of this repository, and name it in your config file
# Loader.py program's _post_load_hook_ method referenced below.  the method is currently hardcoded to read this csv and rename 
# columns to NDA requirements of ['subject', 'subjectkey', 'gender', 'interview_date', 'interview_age']
# as follows.  
        #rosetta = pd.read_csv('UnrelatedHCAHCD_w_STG_Image_and_pseudo_GUID05_27_2020.csv')
        #rosetta = rosetta[['subjectped', 'nda_guid', 'nda_gender', 'nda_interview_date', 'nda_interview_age']]
        #rosetta.columns = ['subject', 'subjectkey', 'gender', 'interview_date', 'interview_age']
#future versions of this code will pull out this file into config.py or even better place, if demand warrants.
#For now, just tweak this function to read your own rosetta file, making sure to result in csv with required, or
# fill out the template file and save it as 'UnrelatedHCAHCD_w_STG_Image_and_pseudo_GUID05_27_2020.csv' or whatever 
# you want it to be under the 'rosetta' attribute in the config file

M.preload_data()

In [ ]:
#Ad hoc functions to clean up empty rows for particular instruments after generated (issue for redcap data)
def redcleanup(structure="lbadl01",filePath="./prepped/hcd/",extraomitcol1='NO',extraomitcol2='NO',extraomitcol3='NO',extraomitcol4='NO',extraomitcol5='NO'):
    print(structure)
    strucroot=structure[:-2]
    strucnum=structure[-2:]

    df=pd.read_csv(filePath+structure+".csv",header=1)
    df.head()

    print("NumRows Before: "+str(df.shape[0]))
    subfields=df.columns.to_list()
    subfields.remove('subjectkey')
    subfields.remove('src_subject_id')
    subfields.remove('interview_date')
    subfields.remove('interview_age')
    subfields.remove('sex')
    try:
       subfields.remove('comqother')
    except:
       pass
    if extraomitcol1 and extraomitcol1 !='NO':
        subfields.remove(extraomitcol1)
    if extraomitcol2 and extraomitcol2 !='NO':
        subfields.remove(extraomitcol2)
    if extraomitcol3 and extraomitcol3 !='NO':
        subfields.remove(extraomitcol3)
    if extraomitcol4 and extraomitcol4 !='NO':
        subfields.remove(extraomitcol4)
    if extraomitcol5 and extraomitcol5 !='NO':
        subfields.remove(extraomitcol5)
    df=df.dropna(how='all',subset=subfields)
    print("NumRows After: "+str(df.shape[0]))

    with open(filePath+structure+".csv",'w') as f:
        f.write(strucroot+","+str(int(strucnum))+"\n")
        df.to_csv(f,index=False)
   
#these guys already set to 99s in map, so null finder wont work above
def asr(structure="asr01",filePath="./prepped/hcd/"):
    print(structure)
    strucroot=structure[:-2]
    strucnum=structure[-2:]
    df=pd.read_csv(filePath+structure+".csv",header=1)
    print("NumRows Before: "+str(df.shape[0]))
    print("NumColumns Before: "+str(df.shape[1]))
    #df=df.loc[df.interview_age>719].copy()
    df=df.loc[~((df.asr2_2==-99)&(df.asr3_2==-99))]
    df=df.drop(columns=['asr1_6_text',
        'asr2_3_text',
        'asr5_5_text',
        'asr7_4_text',
        'asr8_4_text',
        'asr10_6_text',
        'asr12_1_text',                
        'asr13_3_text',
        'asr14_1_text',
        'asr15_2_text',
        'asr15_4_text',
        'asr16_3_text',
        'asr16_4_text',
        'asr17_5_text',
        'asr19_1_text',
        ])
    print("NumRows After: "+str(df.shape[0]))
    print("NumColunms After: "+str(df.shape[1]))
    with open(filePath+structure+".csv",'w') as f:
        f.write(strucroot+","+str(int(strucnum))+"\n")
        df.to_csv(f,index=False)
    #print(subset)

def dropcols(structure="bsc01",filePath="./prepped/hcd/",dropcols=[]):
    print(structure)
    strucroot=structure[:-2]
    strucnum=structure[-2:]
    df=pd.read_csv(filePath+structure+".csv",header=1)
    print("NumColumns Before: "+str(df.shape[1]))
    df=df.drop(columns=dropcols)
    print("NumColumns After: "+str(df.shape[1]))
    with open(filePath+structure+".csv",'w') as f:
        f.write(strucroot+","+str(int(strucnum))+"\n")
        df.to_csv(f,index=False)

        
#these guys already set to 99s in map, so null finder wont work above
def bisbasparent999(structure="bisbas01",filePath="./prepped/hcd/"):
    print(structure)
    strucroot=structure[:-2]
    strucnum=structure[-2:]
    df=pd.read_csv(filePath+structure+".csv",header=1)
    print("NumRows Before: "+str(df.shape[0]))
    df=df.loc[~(df.bissc_total==999)].copy()
    print("NumRows After: "+str(df.shape[0]))
    with open(filePath+structure+".csv",'w') as f:
        f.write(strucroot+","+str(int(strucnum))+"\n")
        df.to_csv(f,index=False)

#these guys already set to 99s in map, so null finder wont work above
def neo999(structure="neo_ffi_form_s_adult_200301",filePath="./prepped/hcd/"):
    print(structure)
    strucroot=structure[:-2]
    strucnum=structure[-2:]
    df=pd.read_csv(filePath+structure+".csv",header=1)
    print("NumRows Before: "+str(df.shape[0]))
    df=df.loc[~((df.neo_n==999)&(df.neo_e==999)&(df.neo_a==999))].copy()
    print("NumRows After: "+str(df.shape[0]))
    with open(filePath+structure+".csv",'w') as f:
        f.write(strucroot+","+str(int(strucnum))+"\n")
        df.to_csv(f,index=False)

def cbcl999(structure="cbcl01",filePath="./prepped/hcd/"):
    print(structure)
    strucroot=structure[:-2]
    strucnum=structure[-2:]
    df=pd.read_csv(filePath+structure+".csv",header=1)
    print("NumRows Before: "+str(df.shape[0]))
    df=df.loc[~((df.cbcl1==999)&(df.cbcl2==999)&(df.cbcl3==999)&(df.cbcl4==999))].copy()
    df=df.drop(columns=['cbcl1_2_text','cbcl2_3_text','cbcl5_2_text','cbcl6_6_text','cbcl7_3_text','cbcl9_2_text','cbcl56h_des','cbcl10_1_text','cbcl11_2_text','cbcl11_6_text','cbcl12_6_text','cbcl13_1_text','cbcl13_5_text','cbcl13_6_text','cbcl13_7_text','cbcl14_7_text','cbcl16_1_text','cbcl16_6_text','cbcl113a_des','cbcl113b_des','cbcl113c_des'])
    print("NumRows After: "+str(df.shape[0]))
    with open(filePath+structure+".csv",'w') as f:
        f.write(strucroot+","+str(int(strucnum))+"\n")
        df.to_csv(f,index=False)

def cbcl1_5_999(structure="cbcl1_501",filePath="./prepped/hcd/"):
    print(structure)
    strucroot=structure[:-2]
    strucnum=structure[-2:]
    df=pd.read_csv(filePath+structure+".csv",header=1)
    print("NumRows Before: "+str(df.shape[0]))
    df=df.loc[~((df.cbcl1==999)&(df.cbcl56a==999)&(df.cbcl_nt==999)&(df.cbcl_eye==999))].copy()
    df=df.drop(columns=['cbcl24b_text','cbcl31b_text','cbcl5_2_text','cbcl7_3_text','cbcl10_1_text','cbcl57b_text','cbcl65b_text','cbcl74b_text','cbcl13_1_text','cbcl13_6_text','cbcl92b_text','cbcl113a_des','cbcl113b_des','cbcl113c_des','cbclpre101_des','adis_concern_what','trf_best'])
    print("NumRows After: "+str(df.shape[0]))
    with open(filePath+structure+".csv",'w') as f:
        f.write(strucroot+","+str(int(strucnum))+"\n")
        df.to_csv(f,index=False)

def psqi(structure="psqi01",filePath="./prepped/hcd/"):
    print(structure)
    strucroot=structure[:-2]
    strucnum=structure[-2:]
    df=pd.read_csv(filePath+structure+".csv",header=1)
    print("NumRows Before: "+str(df.shape[0]))
    df.loc[df.psqip6b_4.isnull()==True,'psqip6b_4']=88
    print("NumRows After: "+str(df.shape[0]))
    with open(filePath+structure+".csv",'w') as f:
        f.write(strucroot+","+str(int(strucnum))+"\n")
        df.to_csv(f,index=False)
    
def medhxdrops(structure="medh01",filePath="./prepped/hcd/"): 
    print(structure)
    strucroot=structure[:-2]
    strucnum=structure[-2:]
    df=pd.read_csv(filePath+structure+".csv",header=1)
    print("NumRows Before: "+str(df.shape[0]))
    df=df.drop(columns=['medhis_6e_describe','medhis_6t_describe','medhis_8b_describe'])
    print("NumRows After: "+str(df.shape[0]))
    with open(filePath+structure+".csv",'w') as f:
        f.write(strucroot+","+str(int(strucnum))+"\n")
        df.to_csv(f,index=False)
    
#these guys already set to 99s in map, so null finder wont work above
def phenx25(structure="phenx_su01",filePath="./prepped/hcd/"):
    print(structure)
    strucroot=structure[:-2]
    strucnum=structure[-2:]
    df=pd.read_csv(filePath+structure+".csv",header=1)
    print("NumRows Before: "+str(df.shape[0]))
    df=df.loc[~((df.ale_total_number_nm==25))].copy()
    print("NumRows After: "+str(df.shape[0]))
    with open(filePath+structure+".csv",'w') as f:
        f.write(strucroot+","+str(int(strucnum))+"\n")
        df.to_csv(f,index=False)

def cleanlist(structurelist=['lbadl01','mchq01']):
    for i in structurelist:
        redcleanup(structure=i,filePath="./prepped/hcd/")
        
def cleanzeros(structure='vitals01',filePath="./prepped/hcd/"):
    #print(structure)
    strucroot=structure[:-2]
    strucnum=structure[-2:]
    df=pd.read_csv(filePath+structure+".csv",header=1)
    df.loc[df.vtl007==0,'vtl007']=np.NaN
    #df.loc[df.bp_stand=='11/80','bp_stand']=np.NaN
    #df.loc[df.bp_stand=='9999','bp_stand']=np.NaN
    with open(filePath+structure+".csv",'w') as f:
        f.write(strucroot+","+str(int(strucnum))+"\n")
        df.to_csv(f,index=False)
        
def integercleanup(structure='asr01',filePath="./prepped/hcd/",varlist=['a']):
    #print(structure)
    strucroot=structure[:-2]
    strucnum=structure[-2:]
    df=pd.read_csv(filePath+structure+".csv",header=1)
    for v in varlist:
        df[v]=df[v].fillna(-9999).astype(int).astype(str).str.replace('-9999','')     
    with open(filePath+structure+".csv",'w') as f:
        f.write(strucroot+","+str(int(strucnum))+"\n")
        df.to_csv(f,index=False)


        

In [ ]:
M.run('ksads_dem01')
#M.run('pcint_from_ksads01')
#M.run('pcint_from_ksadsp201')


In [ ]:
M.run('nffi01')
redcleanup(structure='nffi01',filePath="./prepped/hcd/",
       extraomitcol1='neo2_score_ne',extraomitcol2='neo2_score_ex',extraomitcol3='neo2_score_op',extraomitcol4='neo2_score_ag',extraomitcol5='neo2_score_co')

In [ ]:
M.run('vitals01')
cleanzeros()

In [ ]:
#test one to see if its working
M.run('socdem01')


In [ ]:
#pull in race and ethnicity from code that also derived ndar_subjects, as well as HCD id of parents 
#for the comqother variable
#special clean up of annual_fam_inc vars for this structure
singlepath="~/UbWinSharedSpace1/ccf-nda-behavioral/PycharmToolbox/prepped_singleton_structures/"
racethnic='HCPD_racethnic_for_socdem01_11_10_2020.csv'
re=pd.read_csv(singlepath+racethnic,header=0)

filePath="./prepped/hcd/"
structure='socdem01'
strucroot='socdem'
strucnum='01'
df=pd.read_csv(filePath+structure+".csv",header=1)

redf=pd.merge(re,df,on='src_subject_id',how='inner')
redf.shape
redf.columns
    
redf.loc[redf.annual_fam_inc==9999999,'annual_fam_inc']=-999999  
redf.loc[redf.annual_fam_inc==99999999,'annual_fam_inc']=-999999
redf.loc[redf.annual_fam_inc==9999999999,'annual_fam_inc']=-999999
redf.loc[redf.annual_fam_inc==9999999999999,'annual_fam_inc']=-999999    


In [ ]:
redf.groupby('cg1_ethnicity').count()
redf.loc[redf.cg1_ethnicity.str.contains('Unknown')==True,'cg1_ethnicity']=''

In [ ]:
redf['newcomq']=''
redf.loc[redf.comqother.str.contains('caregiver'),'newcomq']='caregiver '+redf.parent_id+redf.comqother.str.replace('caregiver','')
redf.loc[redf.comqother.str.contains('adult'),'newcomq']=redf.comqother
#redf[['comqother','parent_id','newcomq']]
redf=redf.drop(columns=['comqother','parent_id'])
redf=redf.rename(columns={'newcomq':'comqother'})


In [ ]:
#reorder and rewrite
redf=redf[['src_subject_id','subjectkey', 'interview_date', 'interview_age', 'sex',
       'comqother', 'race', 'ethnicity', 'cg1_race', 'cg1_ethnicity',
       'bio_mother', 'bio_father', 'cust_parent', 'ptner_yn','p_partnerbio',
       'dembadpt', 'sub_adopt_1', 'country_origin', 'cg1_country_origin','ustime','cg1_ustime',
       'nimh_rv_dem_03', 'fspgod',  'cg1_nimh_rv_dem_03', 'cg1_fspgod',
       'das1ms',  'cg1_das1ms',
       'area4_explain', 'employcur', 'paofwork5', 'cg1_area4_explain', 'cg1_employcur','cg1_paofwork5', 
       'ind_type', 'jobh','calm_inc1', 'sub_income','cg1_ind_type', 'dem_industry_mom_12', 'cg1_sub_income',
        'annual_fam_inc', 'sub_income_famcode',
       'household_number_in_house', 'preg_age_mom', 'preg_age_dad',
       'birthcountry_dad', 'birthcountry_mom', 
        'bkgrnd_education',  'mother_edu_cat','father_edu_cat','cg1_bkgrnd_education', 'ptner_grade',
       'family_income_dfct1', 'family_income_dfct2',
       'family_income_dfct3', 'family_income_dfct4', 'family_income_dfct5',
       'family_income_dfct6', 'family_income_dfct7', 
       'ptner_job', 'ptner_job1_1', 'ptner_job7_1',
       'ptner_job8_1', 'ptner_business', 'ptner_work', 'ptner_income']]


In [ ]:
with open(filePath+structure+".csv",'w') as f:
    f.write(strucroot+","+str(int(strucnum))+"\n")
    redf.to_csv(f,index=False)


In [ ]:
# dont rerun socdem, since special fix applied above
structs = [
    'asr01',
    'bsc01',
    'cbcl01',
    'cbcl1_501',
    'cbq01',
    'deldisk01',
    'er4001',
    'leap01',
    'mab01',
    'mctq01',
    'medh01',
    'mendt01',
    'mmse01',
    'phenx_su01',
    'psqi01',
    'saiq01',
    'sdq01',
    'sleepdis01',
#    'socdem01', #specialty structure
    'vision_tests01',
#    'vitals01',
    'wais_iv_part101',
    'wisc_v01',
    'wppsiiv01',
    'ysr01'
]

#parent report as well as self report in some cases so can be multiple rows per person
structs2=[
    'drugscr01',
    'bisbas01',  
    'eatq01',
    'fenvs01',
    'gbi01',
    'pds01',
    'scan_debrief01',
    'srs02',
    'upps01',
    'screentime01'  
]
for s in structs:
    M.run(s)

for s in structs2:
    M.run(s)
#for i in structs:
#    print(i)
#for i in structs2:
#    print(i)


In [ ]:
#now clean up empty rows
cleanlist(structurelist=structs)
cleanlist(structurelist=structs2)
redcleanup(structure="asr01",filePath="./prepped/hcd/",extraomitcol1='somatic_complaints_total',extraomitcol2='missingsum_obvq',extraomitcol3='kksomp') #line is redundant with cleanlist function
asr(structure="asr01",filePath="./prepped/hcd/")
dropcols(structure="bsc01",filePath="./prepped/hcd/",dropcols=['comments'])
bisbasparent999(structure="bisbas01",filePath="./prepped/hcd/")
redcleanup(structure='cbcl01',filePath="./prepped/hcd/",extraomitcol1='version_form')
cbcl999(structure="cbcl01",filePath="./prepped/hcd/")
cbcl1_5_999(structure="cbcl1_501",filePath="./prepped/hcd/")
redcleanup(structure='deldisk01',filePath="./prepped/hcd/",extraomitcol1='version_form')
redcleanup(structure="eatq01",filePath="./prepped/hcd/",extraomitcol1='respond')
redcleanup(structure='gbi01',filePath="./prepped/hcd/",extraomitcol1='version_form',extraomitcol2='sup_y_ss_sum_nm')
redcleanup(structure='mctq01',filePath="./prepped/hcd/",extraomitcol1='version_form',extraomitcol2='frprnts')
#neo999(structure="neo_ffi_form_s_adult_200301",filePath="./prepped/hcd/")
redcleanup(structure="pds01",filePath="./prepped/hcd/",extraomitcol1='respond')
phenx25(structure="phenx_su01",filePath="./prepped/hcd/")
psqi(structure="psqi01",filePath="./prepped/hcd/")
redcleanup(structure='srs02',filePath="./prepped/hcd/",extraomitcol1='respond',extraomitcol2='respond_detail',extraomitcol3='phenotype')
redcleanup(structure='upps01',filePath="./prepped/hcd/",extraomitcol1='version_form')
redcleanup(structure='ysr01',filePath="./prepped/hcd/",extraomitcol1='version_form',extraomitcol2='missingsum_obvq')
ysrcols=['cbcl1_2_text','cbcl2_3_text','cbcl5_2_text','cbcl6_6_text','cbcl7_3_text','cbcl9_2_text','cbcl10_1_text',
    'cbcl11_2_text','cbcl11_6_text','cbcl12_6_text','cbcl13_1_text','cbcl13_5_text','cbcl13_6_text','cbcl13_7_text',
    'cbcl16_1_text','cbcl16_6_text','cbcl56h_des']
dropcols(structure="ysr01",filePath="./prepped/hcd/",dropcols=ysrcols)


In [ ]:
#now fix the non-integer business
#clean up the non-integer business
a=['asr1_2','asr1_4','asr1_6','asr2_1','asr2_2','asr2_3','asr2_4','asr2_5','asr2_6','asr3_2','asr3_3','asr3_4','asr3_5','asr3_6','asr4_1','asr4_2','asr5_3','asr5_5','asr5_6','asr6_1','asr6_2','asr6_3','asr6_5','asr7_2','asr7_4','asr7_5','asr8_2','asr8_3','asr9_1','asr9_2','asr9_3','asr9_4','asr9_5','asr9_6','asr10_1','asr10_3','asr10_4','asr10_5','asr10_6','asr11_1','asr11_2','asr11_3','asr11_4','asr11_5','asr12_1','asr12_3','asr12_5','asr13_1','asr13_2','asr13_5','asr13_6','asr14_1','asr14_4','asr14_5','asr15_1','asr15_3','asr15_4','asr15_5','asr15_6','asr16_1','asr16_2','asr16_3','asr16_4','asr16_5','asr16_6','asr17_1','asr17_3','asr17_4','asr17_5','asr17_6','asr18_1','asr18_2','asr18_3','asr18_4','asr18_5','asr18_6','asr19_1','asr19_3','asr19_4','asr19_5','asr20_1','asr20_2','asr20_4','asr20_5','asr20_6','asr21_1','asr21_2','asr21_4','asr21_5','asr21_6','asr22_4','asr22_6','adh_total','adh_t_score','antisocial_total','antisocial_t_score','anxiety_total','anxiety_t_score','avoidant_total','avoidant_t_score','depressive_total','depressive_t_score','adh_h_i_total','adh_i_total','abcl_critical_total','critical_t_score','asr_syn_csum','abcl_other_total','kksomar','asr_scr_thought_r','asr1_1','asr1_3','asr1_5','asr4_3','asr4_4','asr4_5','asr4_6','asr5_1','asr5_2','asr5_4','asr6_6','asr7_1','asr7_3','asr7_6','asr8_1','asr8_4','asr8_5','asr11_6','asr12_2','asr12_4','asr12_6','asr13_3','asr13_4','asr14_2','asr14_3','asr14_6','asr15_2','asr17_2','asr19_2','asr19_6','asr20_3','asr21_3','asr22_1','asr22_2','asr22_3','asr22_5','asr3_1','asr6_4','asr8_6']
integercleanup(structure='asr01',filePath="./prepped/hcd/",varlist=a)
b=['bisbas2','bisbas3','bisbas4','bisbas5','bisbas7','bisbas8','bisbas9','bisbas10','bisbas12','bisbas13','bisbas14','bisbas15','bisbas16','bisbas18','bisbas19','bisbas20','bisbas23','bisbas24','bisbas_20i','bisbas_16a','bis_y_ss_bas_drive_nm','bis_y_ss_bas_fs_nm','bis_y_ss_bas_rr_nm','bis_y_ss_basm_drive','bis_y_ss_basm_drive_nm','bis_y_ss_basm_rr','bis_y_ss_basm_rr_nm','bis_y_ss_bis_sum_nm','bis_y_ss_bism_sum','bis_y_ss_bism_sum_nm','bisbas1','bisbas6','bisbas11','bisbas17','bisbas21','bisbas22','bisbas_pac1','bisbas_pac2','bisbas_pac3','bisbas_pac4','bisbas_pac5','bisbas_pac6','bisbas_pac7','bisbas_pac8','bisbas_pac9','bisbas_pac10','bisbas_pac11','bisbas_pac12','bisbas_pac13','bisbas_pac14','bisbas_pac15','bisbas_pac16','bisbas_pac17','bisbas_pac18','bisbas_pac19','bisbas_pac20','bas_total']
integercleanup(structure='bisbas01',filePath="./prepped/hcd/",varlist=b)
cc=['cbclpre100','kkwithr','abcl_other_total','cbclpre35_start','cbcl113a','cbcl113b','cbcl113c']
integercleanup(structure='cbcl01',filePath="./prepped/hcd/",varlist=cc)
ccc=['cbclpre100','cbp_4','cbcl_dsm_affectiveproblems_raw','cbcl_dsm_anxietyproblems_raw','cbcl_dsm_pervasive_dev_raw','kkaddr','kkoddr','cbcl_emotionally_reactive_raw']
integercleanup(structure='cbcl1_501',filePath="./prepped/hcd/",varlist=ccc)
cccc=['cbq01','cbq31','cbq59','cbq02','cbq_29','cbq133','cbq_28','cbq124','cbq11','cbq09','cbq_17','cbq49','cbq48','cbq77','cbq71','cbq62','cbq127','cbq69','cbq135','cbq194','cbq03','cbq137','cbq65','cbq54','cbq84','cbq34','cbq191','cbq028','cbq_49','cbq170','cbq147','cbq39','cbq_94','cbq168','cbq33','cbq36']
integercleanup(structure='cbq01',filePath="./prepped/hcd/",varlist=cccc)
c=['ed1_blood','hair_collect','hair2','hair3','hair4','hair5','hair6','hair7','rucdrsaliva','salisaliva_collecttime1','salisaliva_collecttime2','salisaliva_frozen','salisaliva_mestrual','salisaliva_timetofrz','bld_core','bld_core_d2ph','bld_core_d2pm','bld_rucdr']
integercleanup(structure='bsc01',filePath="./prepped/hcd/",varlist=c)
d=['ddisc_valid']
integercleanup(structure='deldisk01',filePath="./prepped/hcd/",varlist=d)
e=['caffeine_s1yn','nicotine_s1yn','drug1_1','drug1_2','drug1_3','drug1_4','drug1_5','drug1_6','caffeine_s1preday','nicotine_s1preday']
integercleanup(structure='drugscr01',filePath="./prepped/hcd/",varlist=e)
ee=['eatq1_1','eatq1_2','eatq1_3','eatq1_4','eatq1_5','eatq1_6','eatq1_7','eatq2_1','eatq2_2','eatq2_3','eatq2_4','eatq2_5','eatq2_6','eatq2_7','eatq3_1','eatq3_2','eatq3_3','eatq3_4','eatq3_5','eatq3_6','eatq3_7','eatq4_1','eatq4_2','eatq4_3','eatq4_4','eatq4_5','eatq4_6','eatq4_7','eatq5_1','eatq5_2','eatq5_3','eatq5_4','eatq5_5','eatq5_6','eatq5_7','eatq6_1','eatq6_2','eatq6_3','eatq6_4','eatq6_5','eatq6_6','eatq6_7','eatq7_1','eatq7_2','eatq7_3','eatq7_4','eatq7_5','eatq7_6','eatq7_7','eatq8_1','eatq8_2','eatq8_3','eatq8_4','eatq8_5','eatq8_6','eatq8_7','eatq9_1','eatq9_2','eatq9_3','eatq9_4','eatq9_5','eatq10_1','eatq10_2','eatq10_3','eatq10_4','epsgbb_004m_a','epacbb_005m_a','epicbb_006m_a','epacbb_017m_a','epafbb_018m_a','epfsbb_020m_a','epfsbb_021m_a','epatbb_022m_a','epagbb_025m_a','epshbb_027m_a','epsgbb_029m_a','epdmbb_033m_a','epsgbb_034m_a','epsgbb_069m_a','epagbb_041m_a','epagbb_042m_a','epicbb_047m_a','epfebb_053m_a','epshbb_054m_a','epfsbb_057m_a','epfebb_061m_a','eatq_p_ss_activation_nm','eatq_p_ss_affiliation_nm','eatq_p_ss_aggression_nm','eatq_p_ss_attention_nm','eatq_p_ss_contl_sup_nm','eatq_p_ss_depressive','eatq_p_ss_depressive_nm','eatq_p_ss_fear_nm','eatq_p_ss_frustration_nm','eatq_p_ss_inhibitory_nm','eatq_p_ss_negaffect_sup_nm','eatq_p_ss_shyness_nm','eatq_p_ss_surgency_nm','eatq_p_ss_surgency_sup_nm']
integercleanup(structure='eatq01',filePath="./prepped/hcd/",varlist=ee)
f=['conflict_y1','conflict_y2','conflict_y3','conflict_y4','conflict_y5','conflict_y6','conflict_y7','conflict_y8','conflict_y9','fes_y_ss_fc','famenv_1','famenv_2','famenv_3','famenv_4','famenv_5','famenv_6','famenv_7','famenv_8','famenv_9']
integercleanup(structure='fenvs01',filePath="./prepped/hcd/",varlist=f)
g=['gbi4','s7upchildself2','gbi6','s7upchildself4','s7upchildself5','s7upchildself6','gbi10','sup_y_ss_sum_nm','sup_y_ss_sum','gbi1','gbi2','gbi3','gbi5','gbi7','gbi8','gbi9','gbitotal']
integercleanup(structure='gbi01',filePath="./prepped/hcd/",varlist=g)
h=['devhis_6_p','devhis_8_prescript_med','devhis_8_tobacco','devhis_8_alcohol','devhis_8_coc_crack','devhis_8_her_morph','devhis_8_other_drugs','demo_preg_meds','devhis_9_other_drugs','devhis_caffeine_11','devhis_caffeine_amt','preg4_2','devhis_10e_p','devhis_10_p','devhis_10i_p','devhis_10j_p','devhis_10k_p','devhis_10m_p','devhis_11_p','ph_11','devhis_14f_p','devhis_14h_p','devhis_15','devhis_16_p','devhis_17_p','devhis_20_p','devhis_21_p','devhis_22_p','devhis_23_p','devhis_9_oxycont','devhis_8_alchohol_max','devhis_8_alchohol_avg','devhis_8_alcohol_effects','devhis_8_cigs_per_day','devhis_8_marijuana_tmswk','devhis_9_prescript_med','devhis_12_p','devhis_9_alchohol_max','devhis_9_alchohol_avg','devhis_9_alcohol_effects','devhis_9_cigs_per_day','devhis_8_other_drugs_amt','devhis_8_other1_tmswk','devhis_9_marijuana_tmswk','devhis_9_other_drugs_amt','devhis_9_other1_tmswk','devhis_8_coc_crack_tmswk']
integercleanup(structure='mab01',filePath="./prepped/hcd/",varlist=h)
hh=['regsch','eo20','wkmnslp','wkmnup','wkalarm','mctq_school8','frmnslp','frmnup','fralarm','frprntslp']
integercleanup(structure='mctq01',filePath="./prepped/hcd/",varlist=hh)
i=['lan_know','lan611','lan612','lan613','lan621','lan622','lan623','lan71','lan72','lan81','lan82','lan631','lan632','lan633','lan73','lan83','lan641','lan642','lan643','lan74','lan84']
integercleanup(structure='leap01',filePath="./prepped/hcd/",varlist=i)
k=['wk_asthma','wk_allergies','seq1c_2','medhis_2d','medhis_2e','cfmh_chd_cerpalsy','baseline_k_005','cfmh_chd_seizure','w1_m_cu_hrng','medhis_2k','ms','medhis_2n','q17_b14','medhis_2p','medhis_2q','medhis_2r','medhis_2s','ph_119','medhx_brokenbones','medhis_6b','medhis_6c','medhis_6d','medhis_6e','medhis_6f','ch_17_00','medhis_6h','ph_neuro9','medhis_6j','medhis_6k','medhis_6l','medhis_6m','medhis_6n','medhis_6o','ph_9','medhis_6q','medhis_6r','medhis_6s','medhis_6t','childhospital','medhis_9a','medhis_4b','ph_9a','medhis_6q_notes','dental1','dental2','dental3','dental4','dental5','dental6']
integercleanup(structure='medh01',filePath="./prepped/hcd/",varlist=k)

l=['num_meds','othmed0a']
integercleanup(structure='mendt01',filePath="./prepped/hcd/",varlist=l)
m=['mmse01','mmse02','mmse03','mmse04','mmse05','mmse06','mmse07','mmse08','mmse09','mmse10','mmse11','mmse12','mmse13','mmse13a','mms14','mms15','mms16','mms17','mms18','mmse14','mmse15','mmse16','mmse17','mmse18','att_calc_tot','alt_tot','mmses4','mmse19','mmse20','mmse21','mmse22','mmse23','mmse24','mmse25','mmse26','mmse27','mmse28','mmse29','mmse30','mmse_ts']
integercleanup(structure='mmse01',filePath="./prepped/hcd/",varlist=m)

n=['nffi_1','nffi_3','nffi_4','nffi_9','nffi_11','nffi_12','nffi_13','nffi_15','nffi_16','nffi_17','nffi_18','nffi_21','nffi_24','nffi_25','nffi_26','nffi_27','nffi_30','nffi_31','nffi_2','nffi_5','nffi_6','nffi_7','nffi_8','nffi_10','nffi_14','nffi_19','nffi_20','nffi_22','nffi_23','nffi_28','nffi_29','nffi_32','nffi_33','nffi_34','nffi_35','nffi_36','nffi_37','nffi_38','nffi_39','nffi_40','nffi_41','nffi_42','nffi_43','nffi_44','nffi_45','nffi_46','nffi_47','nffi_48','nffi_49','nffi_50','nffi_51','nffi_52','nffi_53','nffi_54','nffi_55','nffi_56','nffi_57','nffi_58','nffi_59','nffi_60']
integercleanup(structure='nffi01',filePath="./prepped/hcd/",varlist=n)
pp=['pds_1','pds_2','pds_3','pds_6','pds_7','pufe_005bt','gynecological_hist_6','menstrualcycle4_y','menstrualcycle5_y','menstrualcycle6_y','pds_7b_pv','pds_pv_girl_tanner','pubertyg_19','pubertyg_20','scq_24_a','pds_4','pds_5','pds_pv_boy_tanner','pubertyb_17','pubertyb_18','pds_boy_rs','pds_girl_rs','pds_8_pv']
integercleanup(structure='pds01',filePath="./prepped/hcd/",varlist=pp)
ppp=['fam_mem_died','fam_mem_injured','fam_mem_injured_exp','fam_mem_injured_aff','saw_crime_accident','lost_close_friend','close_friend_sick_injured','parents_fin_situation','fam_mem_drug_alc','sick_injured','parents_argue','mother_father_lostjob','parent_away_home','fam_mem_arrest','close_friend_died','fam_mem_emo_mental','sibling_left_home','sibling_left_home_exp','sibling_left_home_aff','victim_crime_assualt','parents_sep_12mons','parents_trouble_law','attended_new_school','family_moved','parents_divorced','parent_went_to_jail','new_stepparent','parent_new_job','new_sibling','alc_lifetime_use','alc_first_use_age','alc_30day_freq','alc_30day_quantity','cig_smoking','first_use_age_marijuana','frquency_marijuana_code','ale_affect_sum','ale_total_number','lost_close_friend_exp','lost_close_friend_aff','victim_crime_assualt_exp','victim_crime_assualt_aff','parents_sep_12mons_exp','parents_sep_12mons_aff','parents_divorced_exp','parents_divorced_aff','fam_mem_died_exp','fam_mem_died_aff','parent_away_home_exp','parent_away_home_aff','fam_mem_arrest_exp','fam_mem_arrest_aff','attended_new_school_exp','attended_new_school_aff','parent_went_to_jail_exp','parent_went_to_jail_aff','saw_crime_accident_exp','saw_crime_accident_aff','fam_mem_drug_alc_exp','fam_mem_drug_alc_aff','close_friend_died_exp','close_friend_died_aff','fam_mem_emo_mental_exp','fam_mem_emo_mental_aff','family_moved_exp','family_moved_aff','new_stepparent_exp','new_stepparent_aff','parent_new_job_exp','parent_new_job_aff','cig_smoking1_100','cig_smoking1_current','first_cig_smoke_reg_age','cigarette_30day_smoking','freq_30day_count','quantity_30day','first_use_age_sedatives','frquency_sedatives_code','first_use_age_tranq','frquency_tranquilzers_code','first_use_age_painkillers','frquency_painkillers_code','first_use_age_stimulants','frquency_stimulants_code','first_use_age_halluc','frquency_hallucinogens_code','sick_injured_exp','sick_injured_aff','parents_fin_situation_exp','parents_fin_situation_aff','parents_argue_exp','parents_argue_aff','close_friend_sick_injured_exp','close_friend_sick_injured_aff','mother_father_lostjob_exp','mother_father_lostjob_aff','parents_trouble_law_exp','parents_trouble_law_aff','new_sibling_exp','new_sibling_aff','first_use_age_other','frquency_other_code','first_use_age_inhalant','frquency_inhalant_code','first_use_age_cocaine','frquency_cocaine_code','cig_smoking_evryday_1mon','first_cig_smoke_evryday_age1']
integercleanup(structure='phenx_su01',filePath="./prepped/hcd/",varlist=ppp)
pppp=['psqip6a_1','psqip6a_2','psqip6a_3','psqip6a_4','psqip6a_5','psqip6b_1','psqip6b_2','psqip6b_3','psqip6b_4','psqpi8','psqpi7_1','psqpi7_2','psqi_9','psqi_com2_3','psqi_com5','psqi_com7','csq_psqi_2_score','csq_psqi_5a','psqi_component1','psqi_component2','psqi_component6']
integercleanup(structure='psqi01',filePath="./prepped/hcd/",varlist=pppp)
p=['satisfaction1','satisfaction2','satisfaction3','satisfaction4','satisfaction7a','satisfaction8b','satisfaction8','satisfaction9','satisfaction57_1','satisfaction57_10','satisfaction57_2','satisfaction57_3','satisfaction57_4','satisfaction57_7','satisfaction57_8']
integercleanup(structure='scan_debrief01',filePath="./prepped/hcd/",varlist=p)
q=['parent_sleep20','csq_psqi_4_score','insomnia8','psqip6b_5']
integercleanup(structure='psqi01',filePath="./prepped/hcd/",varlist=q)
s=['ccf_sai_p_base_school','ccf_sai_p_base_outside','ccf_sai_p_base_private','ccf_sai_p_base_self','ccf_sai_p_base_nyr','ccf_sai_p_base_nmonth','ccf_sai_p_base_perwk','ccf_sai_p_base_tspent','ccf_sai_p_base_p12','ccf_sai_p_basket_school','ccf_sai_p_basket_outside','ccf_sai_p_basket_private','ccf_sai_p_basket_self','ccf_sai_p_basket_nyr','ccf_sai_p_basket_nmonth','ccf_sai_p_basket_perwk','ccf_sai_p_basket_tspent','ccf_sai_p_basket_p12','ccf_sai_p_soc_school','ccf_sai_p_soc_outside','ccf_sai_p_soc_private','ccf_sai_p_soc_self','ccf_sai_p_soc_nyr','ccf_sai_p_soc_nmonth','ccf_sai_p_soc_perwk','ccf_sai_p_soc_tspent','ccf_sai_p_soc_p12','ccf_sai_p_vball_school','ccf_sai_p_vball_outside','ccf_sai_p_vball_private','ccf_sai_p_vball_self','ccf_sai_p_vball_nyr','ccf_sai_p_vball_nmonth','ccf_sai_p_vball_perwk','ccf_sai_p_vball_tspent','ccf_sai_p_vball_p12','ccf_sai_p_music_school','ccf_sai_p_music_outside','ccf_sai_p_music_private','ccf_sai_p_music_self','ccf_sai_p_music_nyr','ccf_sai_p_music_nmonth','ccf_sai_p_music_perwk','ccf_sai_p_music_tspent','ccf_sai_p_music_p12','ccf_sai_p_gym_school','ccf_sai_p_gym_outside','ccf_sai_p_gym_private','ccf_sai_p_gym_self','ccf_sai_p_gym_nyr','ccf_sai_p_gym_nmonth','ccf_sai_p_gym_perwk','ccf_sai_p_gym_tspent','ccf_sai_p_gym_p12','ccf_sai_p_wpolo_school','ccf_sai_p_wpolo_outside','ccf_sai_p_wpolo_private','ccf_sai_p_wpolo_self','ccf_sai_p_wpolo_nyr','ccf_sai_p_wpolo_nmonth','ccf_sai_p_wpolo_perwk','ccf_sai_p_wpolo_tspent','ccf_sai_p_wpolo_p12','ccf_sai_p_tennis_school','ccf_sai_p_tennis_outside','ccf_sai_p_tennis_private','ccf_sai_p_tennis_self','ccf_sai_p_tennis_nyr','ccf_sai_p_tennis_nmonth','ccf_sai_p_tennis_perwk','ccf_sai_p_tennis_tspent','ccf_sai_p_tennis_p12','ccf_sai_p_dance_school','ccf_sai_p_dance_outside','ccf_sai_p_dance_private','ccf_sai_p_dance_self','ccf_sai_p_dance_nyr','ccf_sai_p_dance_nmonth','ccf_sai_p_dance_perwk','ccf_sai_p_dance_tspent','ccf_sai_p_dance_p12','ccf_sai_p_climb_school','ccf_sai_p_climb_outside','ccf_sai_p_climb_private','ccf_sai_p_climb_self','ccf_sai_p_climb_nyr','ccf_sai_p_climb_nmonth','ccf_sai_p_climb_perwk','ccf_sai_p_climb_tspent','ccf_sai_p_climb_p12','ccf_sai_p_polo_school','ccf_sai_p_polo_outside','ccf_sai_p_polo_private','ccf_sai_p_polo_self','ccf_sai_p_polo_nyr','ccf_sai_p_polo_nmonth','ccf_sai_p_polo_perwk','ccf_sai_p_polo_tspent','ccf_sai_p_polo_p12','ccf_sai_p_iskate_school','ccf_sai_p_iskate_outside','ccf_sai_p_iskate_private','ccf_sai_p_iskate_self','ccf_sai_p_iskate_nyr','ccf_sai_p_iskate_nmonth','ccf_sai_p_iskate_perwk','ccf_sai_p_iskate_tspent','ccf_sai_p_iskate_p12','ccf_sai_p_chess_school','ccf_sai_p_chess_outside','ccf_sai_p_chess_private','ccf_sai_p_chess_self','ccf_sai_p_chess_nyr','ccf_sai_p_chess_nmonth','ccf_sai_p_chess_perwk','ccf_sai_p_chess_tspent','ccf_sai_p_chess_p12','ccf_sai_p_art_school','ccf_sai_p_art_outside','ccf_sai_p_art_private','ccf_sai_p_art_self','ccf_sai_p_art_nyr','ccf_sai_p_art_nmonth','ccf_sai_p_art_perwk','ccf_sai_p_art_tspent','ccf_sai_p_art_p12','ccf_sai_p_drama_school','ccf_sai_p_drama_outside','ccf_sai_p_drama_private','ccf_sai_p_drama_self','ccf_sai_p_drama_nyr','ccf_sai_p_drama_nmonth','ccf_sai_p_drama_perwk','ccf_sai_p_drama_tspent','ccf_sai_p_drama_p12','ccf_sai_p_m_arts_school','ccf_sai_p_m_arts_outside','ccf_sai_p_m_arts_private','ccf_sai_p_m_arts_self','ccf_sai_p_m_arts_nyr','ccf_sai_p_m_arts_nmonth','ccf_sai_p_m_arts_perwk','ccf_sai_p_m_arts_tspent','ccf_sai_p_m_arts_p12','ccf_sai_p_fball_school','ccf_sai_p_fball_outside','ccf_sai_p_fball_private','ccf_sai_p_fball_self','ccf_sai_p_fball_nyr','ccf_sai_p_fball_nmonth','ccf_sai_p_fball_perwk','ccf_sai_p_fball_tspent','ccf_sai_p_fball_p12','ccf_sai_p_crafts_school','ccf_sai_p_crafts_outside','ccf_sai_p_crafts_private','ccf_sai_p_crafts_self','ccf_sai_p_crafts_nyr','ccf_sai_p_crafts_nmonth','ccf_sai_p_crafts_perwk','ccf_sai_p_crafts_tspent','ccf_sai_p_crafts_p12','ccf_sai_p_collect_school','ccf_sai_p_collect_outside','ccf_sai_p_collect_private','ccf_sai_p_collect_self','ccf_sai_p_collect_nyr','ccf_sai_p_collect_nmonth','ccf_sai_p_collect_perwk','ccf_sai_p_collect_tspent','ccf_sai_p_collect_p12','ccf_sai_p_run_school','ccf_sai_p_run_outside','ccf_sai_p_run_private','ccf_sai_p_run_self','ccf_sai_p_run_nyr','ccf_sai_p_run_nmonth','ccf_sai_p_run_perwk','ccf_sai_p_run_tspent','ccf_sai_p_run_p12','ccf_sai_p_mma_school','ccf_sai_p_mma_outside','ccf_sai_p_mma_private','ccf_sai_p_mma_self','ccf_sai_p_mma_nyr','ccf_sai_p_mma_nmonth','ccf_sai_p_mma_perwk','ccf_sai_p_mma_tspent','ccf_sai_p_mma_p12','ccf_sai_p_lax_school','ccf_sai_p_lax_outside','ccf_sai_p_lax_private','ccf_sai_p_lax_self','ccf_sai_p_lax_nyr','ccf_sai_p_lax_nmonth','ccf_sai_p_lax_perwk','ccf_sai_p_lax_tspent','ccf_sai_p_lax_p12','ccf_sai_p_sboard_school','ccf_sai_p_sboard_outside','ccf_sai_p_sboard_private','ccf_sai_p_sboard_self','ccf_sai_p_sboard_nyr','ccf_sai_p_sboard_nmonth','ccf_sai_p_sboard_perwk','ccf_sai_p_sboard_tspent','ccf_sai_p_sboard_p12','ccf_sai_p_skate_school','ccf_sai_p_skate_outside','ccf_sai_p_skate_private','ccf_sai_p_skate_self','ccf_sai_p_skate_nyr','ccf_sai_p_skate_perwk','ccf_sai_p_skate_tspent','ccf_sai_p_skate_p12','ccf_sai_p_yoga_school','ccf_sai_p_yoga_outside','ccf_sai_p_yoga_private','ccf_sai_p_yoga_self','ccf_sai_p_yoga_nyr','ccf_sai_p_yoga_nmonth','ccf_sai_p_yoga_perwk','ccf_sai_p_yoga_tspent','ccf_sai_p_yoga_p12','ccf_sai_p_surf_school','ccf_sai_p_surf_outside','ccf_sai_p_surf_private','ccf_sai_p_surf_self','ccf_sai_p_surf_nyr','ccf_sai_p_surf_nmonth','ccf_sai_p_surf_perwk','ccf_sai_p_surf_tspent','ccf_sai_p_surf_p12','ccf_sai_p_ihock_school','ccf_sai_p_ihock_outside','ccf_sai_p_ihock_private','ccf_sai_p_ihock_self','ccf_sai_p_ihock_nyr','ccf_sai_p_ihock_nmonth','ccf_sai_p_ihock_perwk','ccf_sai_p_ihock_tspent','ccf_sai_p_ihock_p12','ccf_sai_p_skate_nmonth','ccf_sai_p_fhock_school','ccf_sai_p_fhock_outside','ccf_sai_p_fhock_private','ccf_sai_p_fhock_self','ccf_sai_p_fhock_nyr','ccf_sai_p_fhock_nmonth','ccf_sai_p_fhock_perwk','ccf_sai_p_fhock_tspent','ccf_sai_p_fhock_p12','ccf_sai_p_rugby_school','ccf_sai_p_rugby_outside','ccf_sai_p_rugby_private','ccf_sai_p_rugby_self','ccf_sai_p_rugby_nyr','ccf_sai_p_rugby_nmonth','ccf_sai_p_rugby_perwk','ccf_sai_p_rugby_tspent','ccf_sai_p_rugby_p12']
integercleanup(structure='saiq01',filePath="./prepped/hcd/",varlist=s)
ss=['screentime13_y','screentime14_y']
integercleanup(structure='screentime01',filePath="./prepped/hcd/",varlist=ss)
sss=['p_c_sdq_1','p_c_sdq_2','p_c_sdq_3','p_c_sdq_4','p_c_sdq_5','p_c_sdq_6','p_c_sdq_7','p_c_sdq_8','p_c_sdq_9','p_c_sdq_10','p_c_sdq_11','p_c_sdq_12','p_c_sdq_13','p_c_sdq_14','p_c_sdq_15','p_c_sdq_16','p_c_sdq_17','p_c_sdq_18','p_c_sdq_19','p_c_sdq_20','p_c_sdq_21','p_c_sdq_22','p_c_sdq_23','p_c_sdq_24','p_c_sdq_25','p_c_sdq_co_rs','p_c_sdq_em_rs','p_c_sdq_hy_rs','p_c_sdq_pe_rs','p_c_sdq_px_rs','p_c_sdq_so_rs','sdq_pt_externalizing','sdq_pt_internalizing']
integercleanup(structure='sdq01',filePath="./prepped/hcd/",varlist=sss)
ssss=['sleepdisturb_p1','sleepdisturb_p2','sleepdisturb_p3','sleepdisturb_p4','sleepdisturb_p5','sleepdisturb_p6','sleepdisturb_p7','sleepdisturb_p8','sleepdisturb_p9','sleepdisturb_p10','sleepdisturb_p11','sleepdisturb_p12','sleepdisturb_p13','sleepdisturb_p14','sleepdisturb_p15','sleepdisturb_p16','sleepdisturb_p17','sleepdisturb_p18','sleepdisturb_p19','sleepdisturb_p20','sleepdisturb_p21','sleepdisturb_p22','sleepdisturb_p23','sleepdisturb_p24','sleepdisturb_p25','sleepdisturb_p26','sleepdisturb_arsal','sleepdisturb_brth','sleepdisturb_hype','sleepdisturb_mtan','sleepdisturb_somn','sleepdisturb_total','sleepdisturb_trans']
integercleanup(structure='sleepdis01',filePath="./prepped/hcd/",varlist=ssss)
t=['bio_mother','ptner_yn','p_partnerbio','cg1_nimh_rv_dem_03','cg1_employcur','cg1_sub_income','household_number_in_house','preg_age_mom','preg_age_dad','family_income_dfct4','family_income_dfct5','family_income_dfct7','ptner_job1_1','ptner_income','dembadpt','paofwork5','calm_inc1','mother_edu_cat','father_edu_cat','employcur','sub_income_famcode','sub_income','bio_father','cust_parent','ptner_job7_1','cg1_paofwork5',]
integercleanup(structure='socdem01',filePath="./prepped/hcd/",varlist=t)
tt=['parentreport_1','parentreport_2','parentreport_3','parentreport_4','parentreport_5','parentreport_6','parentreport_7','parentreport_8','parentreport_9','parentreport_10','parentreport_11','parentreport_12','parentreport_13','parentreport_14','parentreport_15','parentreport_16','parentreport_17','parentreport_18','parentreport_19','parentreport_20','parentreport_21','parentreport_22','parentreport_23','parentreport_24','parentreport_25','parentreport_26','parentreport_27','parentreport_28','parentreport_29','parentreport_30','parentreport_31','parentreport_32','parentreport_33','parentreport_34','parentreport_35','parentreport_36','parentreport_37','parentreport_38','parentreport_39','parentreport_40','parentreport_41','parentreport_42','parentreport_43','parentreport_44','parentreport_45','parentreport_46','parentreport_47','parentreport_48','parentreport_49','parentreport_50','parentreport_51','srsa52','parentreport_53','parentreport_54','parentreport_55','parentreport_56','parentreport_57','parentreport_58','parentreport_59','parentreport_60','parentreport_61','parentreport_62','parentreport_63','parentreport_64','parentreport_65']
integercleanup(structure='srs02',filePath="./prepped/hcd/",varlist=tt)
u=['upps_p4','upps_p14','upps_p16','upps_p17','upps_p19','upps_p22','upps_p23','upps_p27','upps_p28','upps_p29','upps_p31','upps_p32','upps_p34','upps_p36','upps_p37','upps_p38','upps_p42','upps_p46','upps_p48','upps_p10','upps_p57','upps_p5','upps_p20','upps_plan','phenx_upps_p_1','phenx_upps_p_2','phenx_upps_p_4','phenx_upps_p_7','upps_p18','upps_p21','phenx_upps_p_11','phenx_upps_p_13','phenx_upps_p_14','phenx_upps_p_17','phenx_upps_p_18','phenx_upps_p_20','phenx_upps_p_21','upps_p43','phenx_upps_p_26','phenx_upps_p_27','phenx_upps_p_28','phenx_upps_p_29','phenx_upps_p_30','phenx_upps_p_31','phenx_upps_p_32','phenx_upps_p_33','phenx_upps_p_34','phenx_upps_p_35','phenx_upps_p_36','phenx_upps_p_37','phenx_upps_p_38','phenx_upps_p_39','phenx_upps_p_40']
integercleanup(structure='upps01',filePath="./prepped/hcd/",varlist=u)
uu=['colorvsn_add','colorvsn4','contrastvsn2']
integercleanup(structure='vision_tests01',filePath="./prepped/hcd/",varlist=uu)
w=['subtesttypenr','wais_p','matrixreasoning_1_score','matrixreasoning_2_score','matrixreasoning_3_score','matrixreasoning_4_score','matrixreasoning_5_score','matrixreasoning_6_score','matrixreasoning_7_score','matrixreasoning_8_score','matrixreasoning_9_score','matrixreasoning_10_score','matrixreasoning_11_score','matrixreasoning_12_score','matrixreasoning_13_score','matrixreasoning_14_score','matrixreasoning_15_score','matrixreasoning_16_score','matrixreasoning_17_score','matrixreasoning_18_score','matrixreasoning_19_score','matrixreasoning_20_score','matrixreasoning_21_score','matrixreasoning_22_score','matrixreasoning_23_score','matrixreasoning_24_score','matrixreasoning_25_score','matrixreasoning_26_score']
integercleanup(structure='wais_iv_part101',filePath="./prepped/hcd/",varlist=w)
x=['matrixreason_score','scaled_matrix','subtesttypenr','subtesttypesc','matrixreason_item1','matrixreason_item2','matrixreason_item3','matrixreason_item4','matrixreason_item5','matrixreason_item6','matrixreason_item7','matrixreason_item8','matrixreason_item9','matrixreason_item10','matrixreason_item11','matrixreason_item12','matrixreason_item13','matrixreason_item14','matrixreason_item15','matrixreason_item16','matrixreason_item17','matrixreason_item18','matrixreason_item19','matrixreason_item20','matrixreason_item21','matrixreason_item22','matrixreason_item23','matrixreason_item24','matrixreason_item25','matrixreason_item26','matrixreason_item27','matrixreason_item28','matrixreason_item29','matrixreason_item30','matrixreason_item31','matrixreason_item32']
integercleanup(structure='wisc_v01',filePath="./prepped/hcd/",varlist=x)
y=['wppsi_mxr_rawscre','wppsi_mxr_scldscre','subtesttypenr','wppsi_mxr_p','matrixreason_item1','matrixreason_item2','matrixreason_item3','matrixreason_item4','matrixreason_item5','matrixreason_item6','matrixreason_item7','matrixreason_item8','matrixreason_item9','matrixreason_item10','matrixreason_item11','matrixreason_item12','matrixreason_item13','matrixreason_item14','matrixreason_item15','matrixreason_item16','matrixreason_item17','matrixreason_item18','matrixreason_item19','matrixreason_item20','matrixreason_item21','matrixreason_item22']
integercleanup(structure='wppsiiv01',filePath="./prepped/hcd/",varlist=y)
z=['ysr_1','ysr2','ysr_3','tr04','ysr5','ysr_6','ysr_7','ysr_8','ysr_9','ysr_10','ysr_11','ysr_12','ysr_13','ysr_14','ysr_15','ysr_16','ysr_17','ysr_18','ysr_19','ysr_20','ysr_21','ysr_22','ysr_23','ysr_24','ysr_25','ysr_26','ysr_27','ysr28','ysr_29','ysr_30','ysr_31','ysr_32','ysr_33','ysr_34','ysr_35','ysr_36','ysr_37','ysr_38','ysr_39','ysr_40','ysr_41','ysr_42','ysr_43','ysr_44','ysr_45','ysr_46','ysr_47','ysr_48','ysr_49','ysr_50','ysr_51','ysr_52','ysr_53','ysr_54','ysr_55','ysr_56','ysr_57','ysr_58','ysr_59','ysr_60','ysr_61','ysr_62','ysr_63','ysr_64','ysr_65','ysr_66','ysr_67','ysr_68','ysr_69','ysr_70','ysr_71','ysr_72','ysr_73','ysr_74','ysr_75','ysr_76','ysr_77','ysr_78','ysr_79','ysr_80','ysr_81','ysr_82','ysr_83','ysr_84','tr78','ysr_86','ysr_87','ysr_88','ysr_89','ysr_90','ysr_91','ysr_92','ysr_93','ysr_94','ysr_95','ysr_96','ysr_97','ysr_98','ysr_99','ysr_100','ysr_101','ysr_102','ysr_103','ysr_104','ysr_105','cbcl99','ysr_107','ysr_108','ysr_109','ysr_110','ysr_111','ysr_112','ysr_113','ysr_114','ysr_115','ysr_116','ysr_117','ysr_118','ysr_119','ysra_c_scale','cbcl_adhd','kkdepr','kkanxr','kksomr','kkaddr','kkoddr','kkcdr','kkextr','kkintr','kkanxdr','kkwithr','kksomar','kksocpr','kkthotr','kkattnr','kkdelir','kkaggrr','abcl_other_total','ysra_total_score','cbcl_anxiety','cbcl_conduct','cbcl_oppositional','cbcl_external','cbcl_internal','cbcl_total','anxious_t_score','cbcl_withdrawn_depressed_t_sc','somatic_t_score','cbcl_social_p','thought_t_score','attention_t_score','cbcl_rulebreak','aggressive_t_score','somatic_problems_t_score']
integercleanup(structure='ysr01',filePath="./prepped/hcd/",varlist=z)
